In [6]:
import pandas as pd
import requests

In [13]:
url = "https://api-football-v1.p.rapidapi.com/v3/fixtures"

querystring = {"league":"207","season":"2022"}

headers = {
	"X-RapidAPI-Key": "df439bb8dfmsh328a99e24f9725ep1c783ajsn5976979dfa31",
	"X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'get': 'fixtures', 'parameters': {'league': '207', 'season': '2022'}, 'errors': [], 'results': 182, 'paging': {'current': 1, 'total': 1}, 'response': [{'fixture': {'id': 872530, 'referee': 'S. Schärer', 'timezone': 'UTC', 'date': '2022-07-16T16:00:00+00:00', 'timestamp': 1657987200, 'periods': {'first': 1657987200, 'second': 1657990800}, 'venue': {'id': 10500, 'name': 'Stadion Wankdorf', 'city': 'Bern'}, 'status': {'long': 'Match Finished', 'short': 'FT', 'elapsed': 90}}, 'league': {'id': 207, 'name': 'Super League', 'country': 'Switzerland', 'logo': 'https://media-1.api-sports.io/football/leagues/207.png', 'flag': 'https://media-2.api-sports.io/flags/ch.svg', 'season': 2022, 'round': 'Regular Season - 1'}, 'teams': {'home': {'id': 565, 'name': 'BSC Young Boys', 'logo': 'https://media-3.api-sports.io/football/teams/565.png', 'winner': True}, 'away': {'id': 783, 'name': 'FC Zurich', 'logo': 'https://media-2.api-sports.io/football/teams/783.png', 'winner': False}}, 'goals': {'home': 4, 

In [17]:
data = response.json()

# Extract the fixture data and preprocess it
fixtures = data["response"]
fixture_list = []
for fixture in fixtures:
    fixture_dict = {
        "Heim": fixture["teams"]["home"]["name"],
        "Auswärts": fixture["teams"]["away"]["name"],
        "Ergebnis": f"{fixture['goals']['home']}–{fixture['goals']['away']}",
        "Datum": pd.to_datetime(fixture.get("fixture", {}).get("date")).strftime('%Y-%m-%d %H:%M:%S')
    }
    fixture_list.append(fixture_dict)

df = pd.DataFrame(fixture_list)

# Preprocess the data
df['Ergebnis'] = df['Ergebnis'].apply(lambda x: [int(i) for i in x.split('–')])
df['Heimgewinn'] = df['Ergebnis'].apply(lambda x: int(x[0] > x[1]))
df['Auswärtsgewinn'] = df['Ergebnis'].apply(lambda x: int(x[0] < x[1]))
df['Unentschieden'] = df['Ergebnis'].apply(lambda x: int(x[0] == x[1]))
df['Heim'] = df['Heim'].astype('category')
df['Auswärts'] = df['Auswärts'].astype('category')

# Save the data to the data.csv file
df.to_csv('data.csv', index=False)